![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# Iterative, Distributed Graph Analytics with Pregel

<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/Pregel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


*“Many practical computing problems concern large graphs.”*

Distributed graph processing enables you to do online analytical processing directly on graphs stored in ArangoDB. This is intended to help you gain analytical insights on your data, without having to use external processing systems. Examples of algorithms to execute are PageRank, Vertex Centrality, Vertex Closeness, Connected Components, Community Detection.

Check out the hands-on ArangoDB Pregel Tutorial to learn more.

The processing system inside ArangoDB is based on: Pregel: A System for Large-Scale Graph Processing – Malewicz et al. (Google), 2010. This concept enables us to perform distributed graph processing, without the need for distributed global locking.

This system is not useful for typical online queries, where you just work on a small set of vertices. These kind of tasks are better suited for AQL traversals.

Pregel support since 2014
Predefined algorithms
Could be extended via C++

 https://www.arangodb.com/docs/stable/graphs-pregel.html

https://www.arangodb.com/docs/stable/graphs-pregel.html#available-algorithms



*  Page Rank
*  Seeded PageRank
* Single-Source Shortest Path
* Connected Components:
   * WeaklyConnected
   * StronglyConnected
*Hyperlink-Induced Topic Search (HITS)Permalink
*Vertex Centrality
* Effective Closeness
* LineRank
* Label Propagation
* Speaker-Listener Label Propagation


Best with SMART Graphs https://www.arangodb.com/enterprise-server/smartgraphs/


# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from arango import ArangoClient

Create the temporary database:

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Pregel", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
db = oasis.connect_python_arango(login)

In [ ]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Feel free to use to above URL to checkout the UI!

##  Import Data

In [ ]:
if db.has_graph('school'):
        school = db.graph('school')
else:
        school = db.create_graph('school')

# Retrieve various graph properties.
print(school.name)
print(school.db_name)
print(school.vertex_collections())
print(school.edge_definitions())

In [ ]:
    pregel = db.pregel

    # Start a new Pregel job in "school" graph.
    job_id = db.pregel.create_job(
        graph='school',
        algorithm='pagerank',
        store=False,
        max_gss=100,
        thread_count=1,
        async_mode=False,
        result_field='result',
        algorithm_params={'threshold': 0.000001}
    )

In [ ]:
# Retrieve details of a Pregel job by ID.
job = pregel.job(job_id)
print(job['state']) 

print(job)

In [ ]:
    # Delete a Pregel job by ID.
    pregel.delete_job(job_id)

# Next Steps

To continue playing and working with ArangoDB beyond the temporary database, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLJoin&utm_medium=Github&utm_campaign=ArangoDB%20University)
* Take the [free Graph Course](https://www.arangodb.com/arangodb-graph-course)  
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at https://www.arangodb.com/arangodb-training-center/

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html